In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import pandas as pd
import os
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
%matplotlib inline
from collections import Counter 
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
m = pd.read_csv("/content/gdrive/My Drive/finalmerge.csv",index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
X=m.drop([
               "ArrDelayMinutes",
                "ArrTime",
                "ArrDel15",
                 "Unnamed: 0_x",
                 "Unnamed: 0_y",
                 "Unnamed: 0.1",
                "Origin",
                "Dest",
                "airport_x",
                "airport_y",
                "FlightDate",
                "date_y",
                "date_x",
                "time_x",
                "time_y",
                "windspeedKmph_y",
                "winddirDegree_y",
                "weatherCode_y",
                "precipMM_y",
                "visibility_y",
                "pressure_y",
                "cloudcover_y",
                "DewPointF_y",
                "WindGustKmph_y",
                "tempF_y",
               "WindChillF_y",
               "humidity_y",],axis=1)
X

,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x
0,1,2016,1,1,741.0,0.0,700,0.0,14747,12478,1600,22,255,113,0.0,10,1015,0,32,26,41,32,74
1,1,2016,1,1,705.0,0.0,700,0.0,14747,12892,900,12,31,113,0.0,10,1020,2,30,15,51,47,46
2,1,2016,1,1,658.0,0.0,700,3.0,14747,12892,900,12,31,113,0.0,10,1020,2,30,15,51,47,46
3,1,2016,1,1,723.0,0.0,700,6.0,14747,12478,1500,21,254,113,0.0,10,1016,0,33,24,42,33,74
4,1,2016,1,1,726.0,0.0,700,0.0,14747,12892,1000,10,39,113,0.0,10,1020,9,29,13,53,49,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1815400,3,2017,9,18,2240.0,0.0,2200,0.0,12478,12892,200,5,220,113,0.0,10,1012,1,70,6,74,74,87
1815401,3,2017,9,21,2358.0,1.0,2200,73.0,12478,12892,200,11,179,143,0.0,10,1010,56,63,12,65,65,91
1815402,3,2017,9,22,2240.0,0.0,2200,0.0,12478,12892,200,10,135,113,0.0,10,1010,3,59,12,67,67,75
1815403,3,2017,9,23,2146.0,0.0,2100,3.0,12478,12892,100,6,202,113,0.0,10,1009,4,61,8,68,68,80


In [0]:
Y=m["ArrDel15"]
from sklearn.model_selection import train_test_split  
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.25, random_state = None) 

In [6]:
#Y_train.value_counts()

0.0    1076768
1.0     284785
Name: ArrDel15, dtype: int64

In [0]:
#resampling to make the 0 and 1 values equal 
X = pd.concat([X_train, Y_train], axis=1)

In [0]:
not_delayed = X[X.ArrDel15==0]
delayed = X[X.ArrDel15==1]

In [0]:
from sklearn.utils import resample
delayed_upsampled = resample(delayed,replace=True, n_samples=len(not_delayed),random_state=27)

In [0]:
upsampled = pd.concat([not_delayed, delayed_upsampled])

In [11]:
upsampled.ArrDel15.value_counts()

1.0    1076768
0.0    1076768
Name: ArrDel15, dtype: int64

In [0]:
#values are equal now
X = upsampled.drop(['ArrDel15'], axis=1)
y = upsampled.ArrDel15

In [0]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=0)

In [14]:
X_train

,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x
1002151,4,2017,10,19,1911.0,0.0,1900,3.0,10397,12266,2000,12,108,122,0.0,10,1019,71,62,19,76,76,62
249068,4,2016,12,9,1643.0,1.0,1600,33.0,10397,12889,1700,3,168,113,0.0,10,1023,27,19,5,53,51,27
53089,1,2016,1,10,1138.0,1.0,1100,43.0,13204,11298,1300,12,316,113,0.0,10,1029,0,23,15,37,29,61
48025,1,2016,1,15,845.0,0.0,800,12.0,10397,12889,1000,7,83,122,0.0,10,1016,86,29,9,48,47,55
62728,1,2016,1,6,511.0,0.0,500,0.0,12266,13303,800,24,57,302,6.2,7,1020,100,65,40,67,66,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228301,4,2016,12,31,1018.0,0.0,1000,0.0,12889,12892,1200,8,214,176,0.1,10,1014,100,55,9,58,56,92
924186,1,2017,1,16,1737.0,1.0,1700,22.0,13930,12892,2000,5,232,143,0.0,8,1017,1,63,5,67,66,90
1243805,1,2017,2,8,1334.0,0.0,1300,0.0,13204,11298,1500,28,335,122,0.0,10,1010,33,42,32,64,63,46
1271745,1,2017,2,10,1313.0,1.0,1100,103.0,12478,14771,1500,10,264,122,0.0,10,1018,100,56,13,58,56,93


In [16]:

y_test.value_counts()

0.0    269228
1.0    269156
Name: ArrDel15, dtype: int64

In [17]:
y_train.value_counts()

1.0    807612
0.0    807540
Name: ArrDel15, dtype: int64

In [18]:
#0=1
#decision tree
from sklearn.tree import DecisionTreeClassifier

clf=DecisionTreeClassifier()
clf.fit(X_train,y_train)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
y_pred2 = clf.predict(X_test)

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

         0.0       0.98      0.90      0.94    269228
         1.0       0.91      0.98      0.95    269156

    accuracy                           0.94    538384
   macro avg       0.95      0.94      0.94    538384
weighted avg       0.95      0.94      0.94    538384



In [21]:
#extra tree 
from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier(n_jobs=-1,random_state =None)
etc.fit(X_train,y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [0]:
Y_predetc=etc.predict(X_test)

In [23]:
target_names=["Class 0","Class 1"]
#from sklearn.metrics import classification_report

print(classification_report(y_test,Y_predetc,target_names=target_names))

              precision    recall  f1-score   support

     Class 0       0.99      0.94      0.96    269228
     Class 1       0.94      0.99      0.96    269156

    accuracy                           0.96    538384
   macro avg       0.96      0.96      0.96    538384
weighted avg       0.96      0.96      0.96    538384



In [24]:
Y_predetc
xtdf=pd.DataFrame(Y_predetc)
xtdf[0]
X_test["ArrDelayMinutes"]=m["ArrDelayMinutes"]
X_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID,CRSArrTime,windspeedKmph_x,winddirDegree_x,weatherCode_x,precipMM_x,visibility_x,pressure_x,cloudcover_x,DewPointF_x,WindGustKmph_x,tempF_x,WindChillF_x,humidity_x,ArrDelayMinutes
1685804,3,2017,8,14,1436.0,1.0,1400,36.0,14747,14771,1600,18,258,122,0.0,10,1011,100,58,21,62,61,90,33.0
754837,3,2016,7,11,1042.0,0.0,1000,0.0,14771,11298,1600,27,182,200,0.0,10,1011,50,71,31,95,95,53,0.0
1254628,1,2017,2,20,2022.0,1.0,1800,147.0,13204,12892,2000,5,108,143,0.1,2,1025,100,57,6,51,50,93,145.0
1534758,2,2017,6,20,850.0,1.0,800,15.0,12889,12892,900,8,235,113,0.0,10,1011,0,64,10,65,67,89,16.0
987516,1,2017,1,3,829.0,1.0,800,26.0,13303,14771,1100,20,148,302,1.5,7,1016,100,49,24,54,48,90,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510463,2,2016,4,20,1208.0,1.0,800,206.0,11057,11292,1000,8,196,113,0.0,10,1016,2,31,9,43,42,60,196.0
636970,2,2016,6,20,800.0,0.0,800,0.0,12889,12892,900,5,298,113,0.0,10,1011,15,63,9,81,76,64,24.0
695921,3,2016,7,23,731.0,0.0,700,1.0,11057,12478,900,15,267,113,0.0,10,1010,0,73,21,87,84,71,25.0
264699,4,2016,12,30,1803.0,1.0,1700,38.0,11292,10397,2200,6,249,113,0.0,10,1024,0,27,7,38,38,43,21.0


In [0]:
import collections

classified_df = pd.DataFrame(X_test)

classified_df['Arrdel15']=Y_predetc

In [0]:
classified_df
classified_df.to_csv("mod3manipulation.csv")

In [0]:
from google.colab import files
files.download("mod3manipulation.csv")

In [30]:
#xgboost
import xgboost 
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
#not able to fit the model in xgboost and logistic since i dropped arraydelayminutes and arrdel15
#taking the csv from extra trees as i got the highest accuracy in that
